# Deconfounder in Pyro
This it the Pyro implementation of the Deconfounder from Blei-labs "Blessings of Multiple Causes"

In [1]:
import numpy as np
import pandas as pd
import pyro
import torch
from scipy import sparse
from sklearn.model_selection import train_test_split
pyro.set_rng_seed(101)



import matplotlib
matplotlib.rcParams.update({'font.sans-serif' : 'Helvetica',
                            'axes.labelsize': 10,
                            'xtick.labelsize' : 6,
                            'ytick.labelsize' : 6,
                            'axes.titlesize' : 10})
import matplotlib.pyplot as plt

import seaborn as sns
color_names = ["windows blue",
               "amber",
               "crimson",
               "faded green",
               "dusty purple",
               "greyish"]
colors = sns.xkcd_palette(color_names)
sns.set(style="white", palette=sns.xkcd_palette(color_names), color_codes = False)

In [3]:
X_all = pd.read_csv("ohe_movies.csv", index_col = 0)

# Remove title and revenue
X_all = X_all[X_all.columns.difference(['title_x', 'revenue'])]

x_train = X_all

## Probabilistic PCA model:

\begin{equation}
\mathbf{z}_{n} \stackrel{iid}{\sim} N(\mathbf{0}, \mathbf{I}_K) \\
\mathbf{w}_{n} \stackrel{iid}{\sim} N(\mathbf{0}, \mathbf{I}_K) \\
\mathbf{x}_n \mid \mathbf{z}_n
\sim Bernoulli(\mathbf{z}_n\mathbf{W})
\end{equation}

In [4]:
# Convert input from dataframe to tensor

x_train_tensors = torch.tensor(x_train.to_numpy(dtype = 'float32'))

In [5]:
def ppca_model(data):
    '''
    the probabilistic PCA model
    '''
    num_datapoints, data_dim = data.shape
    
    latent_dim = 2 # This can be varied
    
    # Parameters for w
    w_mean0 = torch.zeros([latent_dim, data_dim])
    w_std0 = torch.ones([latent_dim, data_dim])
    
    # Parameters for z
    z_mean0 = torch.zeros([num_datapoints, latent_dim])
    z_std0 = torch.ones([num_datapoints, latent_dim])


    w = pyro.sample("w", pyro.distributions.Normal(loc = w_mean0, 
                                                   scale = w_std0))

    z = pyro.sample("z", pyro.distributions.Normal(loc = z_mean0, 
                                                   scale = z_std0))

    linear_exp = torch.matmul(z, w)


    # observe data using the bernoulli likelihood
    
    x = pyro.sample("x", pyro.distributions.Bernoulli(logits = linear_exp), obs = data)

In [6]:
def guide(data):
    '''
    guide function to be used for SVI
    '''
    num_datapoints, data_dim = data.shape
    
    latent_dim = 2 # This can be varied
    
    # To infer w params
    
    qw_mean = pyro.param("qw_mean", torch.zeros([latent_dim, data_dim]))
    qw_stddv = pyro.param("qw_stddv", torch.ones([latent_dim, data_dim]),
                         constraint=constraints.positive)
    
    # To infer z params
    
    qz_mean = pyro.param("qz_mean", torch.zeros([num_datapoints, latent_dim]))
    qz_stddv = pyro.param("qz_stddv", torch.ones([num_datapoints, latent_dim]),
                         constraint=constraints.positive)
    
    w = pyro.sample("w", pyro.distributions.Normal(loc = qw_mean, 
                                                   scale = qw_stddv))
    z = pyro.sample("z", pyro.distributions.Normal(loc = qz_mean, 
                                                   scale = qz_stddv))

## SVI
Takes 45-50 minutes for the entire dataset, but much faster than HMC.

In [7]:
from pyro.infer import SVI, Trace_ELBO
import torch.distributions.constraints as constraints
from pyro.optim import Adam

adam_params = {"lr": 0.0005}
optimizer = Adam(adam_params)

svi = SVI(ppca_model, guide, optimizer, loss=Trace_ELBO())

n_steps = 2000

# do gradient steps
for step in range(n_steps):
    svi.step(x_train_tensors)
    if step % 100 == 0:
        print('.', end='')

....................

In [8]:
# grab the learned variational parameters
qw_m = pyro.param("qw_mean")
qw_s = pyro.param("qw_stddv")

qz_m = pyro.param("qz_mean")
qz_s = pyro.param("qz_stddv")

In [9]:
inferred_z = z = pyro.sample("inf_z", pyro.distributions.Normal(loc = qz_m, 
                                                   scale = qz_s))

In [10]:
inferred_z

tensor([[-0.2490,  0.1326],
        [ 0.1928, -0.2690],
        [-0.5804,  0.0489],
        ...,
        [-0.9204, -0.8885],
        [ 0.0867, -0.1378],
        [ 0.1039, -1.3772]], grad_fn=<AddBackward0>)

In [11]:
data = pd.read_csv("ohe_movies.csv", index_col = 0)

In [12]:
data.head()

,title_x,revenue,Larry Mullen Jr.,50 Cent,A. J. Benza,A. J. Langer,A. Jay Radcliff,A. Michael Baldwin,A. Russell Andrews,A.D. Miles,...,Zuzana Geislerová,Ángel Salazar,Édgar Vivar,Ólafur Darri Ólafsson,Óscar Casas,Óscar Jaenada,Ørjan Gamst,Đỗ Thị Hải Yến,Юлия Снигирь,徐帆
0,Avatar,2787965087,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Pirates of the Caribbean: At World's End,961000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Spectre,880674609,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Dark Knight Rises,1084939099,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,John Carter,284139100,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
z = np.array(inferred_z.tolist())

In [19]:
data['Z1'] = z[:, 0]
data['Z2'] = z[:, 1]

In [20]:
data.to_csv('ohe_movies_confounders.csv', header=True, index=False)

## HMC
Really slow but works without a guide function. This attempt is only with 500 columns and not all 13852 columns.

In [73]:
from pyro.infer.mcmc import HMC, MCMC
# from pyro.infer import mcmc

hmc_kernel = HMC(ppca_model, step_size=0.0855, num_steps=4)

x_new = x_train_tensors[:, :500]

ppca_model(x_new)

mcmc_run = MCMC(hmc_kernel, num_samples=500, warmup_steps=100).run(x_new)

Sample: 100%|███████████████████████████████████| 600/600 [04:05<00:00,  4.62it/s, step size=8.63e-02, acc. rate=0.857]


In [75]:
from pyro.infer import EmpiricalMarginal
posterior_z = EmpiricalMarginal(mcmc_run, 'z')

In [93]:
posterior_z.mean

tensor([[ 0.4385, -2.2742],
        [-0.0849, -2.0328],
        [-0.0924, -1.9028],
        ...,
        [ 0.2301, -2.0688],
        [ 1.1267, -1.4036],
        [ 0.1130, -2.0810]])